In [2]:
# define vectorized versions of the kernel functions
# like 1000x faster at assembling covariance functions

import tensorflow as tf
import numpy as np
Lx = 0.4; Ly = 0.4
DTYPE = tf.float32

L_tf = tf.constant(np.array([Lx, Ly]), dtype=DTYPE)
def kernel2d_tf(x, xp):
    # x input is N x 1 x 2 array, xp is 1 x M x 2 array
    # xbar is then an N x M x 2 shape array
    # print(f"{x=} {L_tf=}")
    xbar = (x - xp) / L_tf
    # output is N x M matrix of kernel matrix
    return tf.exp(-0.5 * tf.reduce_sum(tf.pow(xbar, 2.0), axis=-1))

def d_fact(xbar, L):
    return L**(-1.0) * (-xbar)

def d2_fact(xbar, L):
    return L**(-2.0) * (-1.0 + xbar**2)

def d4_fact(xbar,L):
    return L**(-4.0) * (3.0 - 6.0 * xbar**2 + xbar**4)

def d6_fact(xbar,L):
    return L**(-6.0) * (-15 + 45 * xbar**2 - 15 * xbar**4 + xbar**6)

def d8_fact(xbar,L):
    return L**(-8.0) * (105 - 420 * xbar**2 + 210 * xbar**4 - 28 * xbar**6 + xbar**8)

def dx2_kernel_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    xbar = (x - xp) / L_tf # N x M x 2 matrix
    x1bar = xbar[:,:,0]
    x2bar = xbar[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)

    # TODO : shear or general case later (just axial now)
    return K * (d2_fact(x1bar, Lx))

def dy2_kernel_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    xbar = (x - xp) / L_tf # N x M x 2 matrix
    x1bar = xbar[:,:,0]
    x2bar = xbar[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)
    return K * (d2_fact(x2bar, Ly))

def doubledx2_kernel_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    xbar = (x - xp) / L_tf # N x M x 2 matrix
    x1bar = xbar[:,:,0]
    x2bar = xbar[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)

    # TODO : shear or general case later (just axial now)
    return K * (d4_fact(x1bar, Lx))

def doubledy2_kernel_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    xbar = (x - xp) / L_tf # N x M x 2 matrix
    x1bar = xbar[:,:,0]
    x2bar = xbar[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)
    return K * (d4_fact(x2bar, Ly))

def kernel2d_bilapl_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    xbar = (x - xp) / L_tf # N x M x 2 matrix
    x1bar = xbar[:,:,0]
    x2bar = xbar[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)

    return K * (d4_fact(x1bar,Lx) + 2.0 * d2_fact(x1bar, Lx) * d2_fact(x2bar, Ly) + d4_fact(x2bar, Ly))

def kernel2d_double_bilapl_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    xbar = (x - xp) / L_tf # N x M x 2 matrix
    x1bar = xbar[:,:,0]
    x2bar = xbar[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)

    return K * (d8_fact(x1bar,Lx) + \
                4.0 * d6_fact(x1bar, Lx) * d2_fact(x2bar, Ly) +\
                6.0 * d4_fact(x1bar, Lx) * d4_fact(x2bar, Ly) +\
                4.0 * d2_fact(x1bar, Lx) * d6_fact(x2bar, Ly) +\
                d8_fact(x2bar, Ly))

def dx2_bilapl_kernel_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    xbar = (x - xp) / L_tf # N x M x 2 matrix
    x1bar = xbar[:,:,0]
    x2bar = xbar[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)

    # TODO : shear or general case later (just axial now)
    return K * \
        (d6_fact(x1bar,Lx) + 2.0 * d4_fact(x1bar, Lx) * d2_fact(x2bar, Ly) + d2_fact(x1bar, Lx) * d4_fact(x2bar, Ly))

def dy2_bilapl_kernel_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    xbar = (x - xp) / L_tf # N x M x 2 matrix
    x1bar = xbar[:,:,0]
    x2bar = xbar[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)
    return K * \
        (d4_fact(x1bar,Lx) * d2_fact(x2bar, Ly) + 2.0 * d2_fact(x1bar, Lx) * d4_fact(x2bar, Ly) + d6_fact(x2bar, Ly))

def dx2_dy2_kernel_tf(x, xp):
    # first two dimensions are NxN matrix parts
    # x is N x 1 x 2 matrix, xp is 1 x M x 2 matrix
    xbar = (x - xp) / L_tf # N x M x 2 matrix
    x1bar = xbar[:,:,0]
    x2bar = xbar[:,:,1]
    Lx = L_tf[0]; Ly = L_tf[1]

    # baseline kernel matrix which we will scale and add some terms
    K = kernel2d_tf(x,xp)
    return K * d2_fact(x1bar, Lx) * d2_fact(x2bar, Ly)

In [28]:
# verify the kernel function derivatives with tensorflow gradient taping..
import numpy as np
import tensorflow as tf
xy = np.random.rand(1,1,2).astype(np.float32)
x = tf.constant(xy[:,:,0:1]); y = tf.constant(xy[:,:,1:2])
# temp = tf.concat([x, y], axis=0)
xyp = tf.constant(np.random.rand(1,1,2).astype(np.float32))
xp = tf.constant(xyp[:, :, 0:1]); yp = tf.constant(xyp[:, :, 1:2])

with tf.GradientTape(persistent=True) as tape4:
    tape4.watch(x)
    tape4.watch(y)
    with tf.GradientTape(persistent=True) as tape3:
        tape3.watch(x)
        tape3.watch(y)
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(x)
            tape2.watch(y)
            with tf.GradientTape(persistent=True) as tape1:
                tape1.watch(x)
                tape1.watch(y)
                K = kernel2d_tf(tf.concat([x, y], axis=2), xyp)
                dKdx = tape1.gradient(K, x)
                dKdy = tape1.gradient(K, y)
                # print(f"{K=} {dKdx=} {dKdy=}")
            dx2 = tape2.gradient(dKdx, x)
            dy2 = tape2.gradient(dKdy, y)
            print(f"{dx2=} {dy2=}")
        dx3 = tape3.gradient(dx2, x)
        dxxy = tape3.gradient(dx2, y)
        dy3 = tape3.gradient(dy2, y)
    dx4 = tape4.gradient(dx3, x)
    dx2y2 = tape4.gradient(dxxy, y)
    dy4 = tape4.gradient(dy3, y)

del tape1, tape2, tape3, tape4

# test dx2 k_x derivs
analytic_dx2 = dx2_kernel_tf(xy, xyp).numpy()[0,0]
dx2_val = dx2.numpy()[0,0,0]
dx2_rel_err = abs((analytic_dx2 - dx2_val) / dx2_val)
print(f"{dx2_val=} {analytic_dx2=} {dx2_rel_err=}")

# test dy2 k_x derivs
analytic_dy2 = dy2_kernel_tf(xy, xyp).numpy()[0,0]
dy2_val = dy2.numpy()[0,0,0]
dy2_rel_err = abs((analytic_dy2 - dy2_val) / dy2_val)
print(f"{dy2_val=} {analytic_dy2=} {dy2_rel_err=}")

# the nabla^4_x k_x for 2d kernel seems to be correct!
tf_bilapl_x = (dx4 + 2.0 * dx2y2 + dy4).numpy()[0,0,0]
analytic_bilapl_x = kernel2d_bilapl_tf(xy, xyp).numpy()[0,0]
bilapl_rel_err = np.abs((tf_bilapl_x - analytic_bilapl_x) / tf_bilapl_x)
print(f"{tf_bilapl_x=} {analytic_bilapl_x=} {bilapl_rel_err=}")

dx2=<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[-3.4815216]]], dtype=float32)> dy2=<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[0.4954788]]], dtype=float32)>
dx2_val=-3.4815216 analytic_dx2=-3.4815216 dx2_rel_err=0.0
dy2_val=0.4954788 analytic_dy2=0.495479 dy2_rel_err=3.608912e-07
tf_bilapl_x=2.608574 analytic_bilapl_x=2.6085684 bilapl_rel_err=2.1021553e-06


In [33]:
# check dx2, dy2 cross derivs here (4th order)
with tf.GradientTape(persistent=True) as tape4:
    tape4.watch(xp)
    tape4.watch(yp)
    with tf.GradientTape(persistent=True) as tape3:
        tape3.watch(xp)
        tape3.watch(yp)
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(x)
            tape2.watch(y)
            with tf.GradientTape(persistent=True) as tape1:
                tape1.watch(x)
                tape1.watch(y)
                K = kernel2d_tf(tf.concat([x, y], axis=2), tf.concat([xp, yp], axis=2))
                dKdx = tape1.gradient(K, x)
                dKdy = tape1.gradient(K, y)
                # print(f"{K=} {dKdx=} {dKdy=}")
            dx2 = tape2.gradient(dKdx, x)
            dy2 = tape2.gradient(dKdy, y)
            # print(f"{dx2=} {dy2=}")
        dx2_dxp = tape3.gradient(dx2, xp)
        dy2_dyp = tape3.gradient(dy2, yp)
        dx2_dyp = tape3.gradient(dx2, yp)
    dx2_dxp2 = tape4.gradient(dx2_dxp, xp)
    dy2_dyp2 = tape4.gradient(dy2_dyp, yp)
    dx2_dyp2 = tape4.gradient(dx2_dyp, yp)

# test double dx2 k_x derivs
analytic_doubledx2 = doubledx2_kernel_tf(xy, xyp).numpy()[0,0]
doubledx2_val = dx2_dxp2.numpy()[0,0,0]
doubledx2_rel_err = abs((analytic_doubledx2 - doubledx2_val) / doubledx2_val)
print(f"{doubledx2_val=} {analytic_doubledx2=} {doubledx2_rel_err=}")

# test double dy2 k_x derivs
analytic_doubledy2 = doubledy2_kernel_tf(xy, xyp).numpy()[0,0]
doubledy2_val = dy2_dyp2.numpy()[0,0,0]
doubledy2_rel_err = abs((analytic_doubledy2 - doubledy2_val) / doubledy2_val)
print(f"{doubledy2_val=} {analytic_doubledy2=} {doubledy2_rel_err=}")

# test mixed dx2 dyp2 derivs
analytic_dx2_dyp2 = dx2_dy2_kernel_tf(xy, xyp).numpy()[0,0]
dx2_dyp2_val = dx2_dyp2.numpy()[0,0,0]
dx2_dyp2_rel_err = abs((analytic_dx2_dyp2 - dx2_dyp2_val) / dx2_dyp2_val)
print(f"{analytic_dx2_dyp2=} {dx2_dyp2_val=} {dx2_dyp2_rel_err=}")

doubledx2_val=64.639046 analytic_doubledx2=64.639046 doubledx2_rel_err=0.0
doubledy2_val=-55.897263 analytic_doubledy2=-55.897266 doubledy2_rel_err=6.8244795e-08
analytic_dx2_dyp2=-3.066605 dx2_dyp2_val=-3.0666041 dx2_dyp2_rel_err=3.109871e-07


In [37]:
# check 6th order derivs for cross-covariance btw dx2, dy2 and bilapl xyp
with tf.GradientTape(persistent=True) as tape6:
    tape6.watch(xp)
    tape6.watch(yp)
    with tf.GradientTape(persistent=True) as tape5:
        tape5.watch(xp)
        tape5.watch(yp)
        with tf.GradientTape(persistent=True) as tape4:
            tape4.watch(x)
            tape4.watch(y)
            with tf.GradientTape(persistent=True) as tape3:
                tape3.watch(x)
                tape3.watch(y)
                with tf.GradientTape(persistent=True) as tape2:
                    tape2.watch(x)
                    tape2.watch(y)
                    with tf.GradientTape(persistent=True) as tape1:
                        tape1.watch(x)
                        tape1.watch(y)
                        K = kernel2d_tf(tf.concat([x, y], axis=2), tf.concat([xp, yp], axis=2))
                        dKdx = tape1.gradient(K, x)
                        dKdy = tape1.gradient(K, y)
                        # print(f"{K=} {dKdx=} {dKdy=}")
                    dx2 = tape2.gradient(dKdx, x)
                    dy2 = tape2.gradient(dKdy, y)
                    # print(f"{dx2=} {dy2=}")
                dx3 = tape3.gradient(dx2, x)
                dxxy = tape3.gradient(dx2, y)
                dy3 = tape3.gradient(dy2, y)
            dx4 = tape4.gradient(dx3, x)
            dx2y2 = tape4.gradient(dxxy, y)
            dy4 = tape4.gradient(dy3, y)
            bilapl = dx4 + 2.0 * dx2y2 + dy4
        bilapl_dxp = tape5.gradient(bilapl, xp)
        bilapl_dyp = tape5.gradient(bilapl, yp)
    bilapl_dxp2 = tape5.gradient(bilapl_dxp, xp)
    bilapl_dyp2 = tape5.gradient(bilapl_dyp, yp)
        
# test bilapl with dxp2 k_x derivs
analytic_bilapl_dxp2 = dx2_bilapl_kernel_tf(xy, xyp).numpy()[0,0]
bilapl_dxp2_val = bilapl_dxp2.numpy()[0,0,0]
dx2_bilapl_rel_err = abs((analytic_bilapl_dxp2 - bilapl_dxp2_val) / bilapl_dxp2_val)
print(f"{bilapl_dxp2_val=} {analytic_bilapl_dxp2=} {dx2_bilapl_rel_err=}")

# test bilapl with dxp2 k_x derivs
analytic_bilapl_dyp2 = dy2_bilapl_kernel_tf(xy, xyp).numpy()[0,0]
bilapl_dyp2_val = bilapl_dyp2.numpy()[0,0,0]
dy2_bilapl_rel_err = abs((analytic_bilapl_dyp2 - bilapl_dyp2_val) / bilapl_dyp2_val)
print(f"{bilapl_dyp2_val=} {analytic_bilapl_dyp2=} {dy2_bilapl_rel_err=}")

bilapl_dxp2_val=-1540.2228 analytic_bilapl_dxp2=-1540.2225 dx2_bilapl_rel_err=1.5850995e-07
bilapl_dyp2_val=3262.2231 analytic_bilapl_dyp2=3262.2227 dy2_bilapl_rel_err=1.4967745e-07


In [38]:
# verify 8th order derivatives

# now check the bilaplacian with 8th order derivatives..
# namely nabla_x^4 \odot nabla_{x'}^4 \odot K(x,x') with x,x' in R^2
with tf.GradientTape(persistent=True) as tape8:
    tape8.watch(xp)
    tape8.watch(yp)
    with tf.GradientTape(persistent=True) as tape7:
        tape7.watch(xp)
        tape7.watch(yp)
        with tf.GradientTape(persistent=True) as tape6:
            tape6.watch(xp)
            tape6.watch(yp)
            with tf.GradientTape(persistent=True) as tape5:
                tape5.watch(xp)
                tape5.watch(yp)
                with tf.GradientTape(persistent=True) as tape4:
                    tape4.watch(x)
                    tape4.watch(y)
                    with tf.GradientTape(persistent=True) as tape3:
                        tape3.watch(x)
                        tape3.watch(y)
                        with tf.GradientTape(persistent=True) as tape2:
                            tape2.watch(x)
                            tape2.watch(y)
                            with tf.GradientTape(persistent=True) as tape1:
                                tape1.watch(x)
                                tape1.watch(y)
                                K = kernel2d_tf(tf.concat([x, y], axis=2), tf.concat([xp, yp], axis=2))
                                dKdx = tape1.gradient(K, x)
                                dKdy = tape1.gradient(K, y)
                                # print(f"{K=} {dKdx=} {dKdy=}")
                            dx2 = tape2.gradient(dKdx, x)
                            dy2 = tape2.gradient(dKdy, y)
                            # print(f"{dx2=} {dy2=}")
                        dx3 = tape3.gradient(dx2, x)
                        dxxy = tape3.gradient(dx2, y)
                        dy3 = tape3.gradient(dy2, y)
                    dx4 = tape4.gradient(dx3, x)
                    dx2y2 = tape4.gradient(dxxy, y)
                    dy4 = tape4.gradient(dy3, y)
                dxp_dx4 = tape5.gradient(dx4, xp)
                dxp_dx2y2 = tape5.gradient(dx2y2, xp)
                dxp_dy4 = tape5.gradient(dy4, xp)
                dyp_dx4 = tape5.gradient(dx4, yp)
                dyp_dx2y2 = tape5.gradient(dx2y2, yp)
                dyp_dy4 = tape5.gradient(dy4, yp)
            dxp2_dx4 = tape6.gradient(dxp_dx4,xp)
            dyp2_dx4 = tape6.gradient(dyp_dx4, yp)
            dxp2_dx2y2 = tape6.gradient(dxp_dx2y2,xp)
            dyp2_dx2y2 = tape6.gradient(dyp_dx2y2, yp)
            dxp2_dy4 = tape6.gradient(dxp_dy4,xp)
            dyp2_dy4 = tape6.gradient(dyp_dy4, yp)
        dxp3_dx4 = tape7.gradient(dxp2_dx4, xp)
        dxpxpyp_dx4 = tape7.gradient(dxp2_dx4, yp)
        dyp3_dx4 = tape7.gradient(dyp2_dx4, yp)
        dxp3_dx2y2 = tape7.gradient(dxp2_dx2y2, xp)
        dxpxpyp_dx2y2 = tape7.gradient(dxp2_dx2y2, yp)
        dyp3_dx2y2 = tape7.gradient(dyp2_dx2y2, yp)
        dxp3_dy4 = tape7.gradient(dxp2_dy4, xp)
        dxpxpyp_dy4 = tape7.gradient(dxp2_dy4, yp)
        dyp3_dy4 = tape7.gradient(dyp2_dy4, yp)
    dxp4_dx4 = tape8.gradient(dxp3_dx4, xp)
    dxp2yp2_dx4 = tape8.gradient(dxpxpyp_dx4, yp)
    dyp4_dx4 = tape8.gradient(dyp3_dx4, yp)
    dxp4_dx2y2 = tape8.gradient(dxp3_dx2y2, xp)
    dxp2yp2_dx2y2 = tape8.gradient(dxpxpyp_dx2y2, yp)
    dyp4_dx2y2 = tape8.gradient(dyp3_dx2y2, yp)
    dxp4_dy4 = tape8.gradient(dxp3_dy4, xp)
    dxp2yp2_dy4 = tape8.gradient(dxpxpyp_dy4, yp)
    dyp4_dy4 = tape8.gradient(dyp3_dy4, yp)

tf_bilaplx_bilaplxp = dxp4_dx4 + 2.0 * dxp4_dx2y2 + dxp4_dy4 + \
                      2.0 * (dxp2yp2_dx4 + 2.0 * dxp2yp2_dx2y2 + dxp2yp2_dy4) + \
                      dyp4_dx4 + 2.0 * dyp4_dx2y2 + dyp4_dy4
analytic_bilaplx_bilaplxp = kernel2d_double_bilapl_tf(xy, xyp)
double_bilapl_rel_err = abs((tf_bilaplx_bilaplxp-analytic_bilaplx_bilaplxp)/analytic_bilaplx_bilaplxp)


print(f"{tf_bilaplx_bilaplxp=}")
print(f"{analytic_bilaplx_bilaplxp=}")
print(f"{double_bilapl_rel_err=}")


tf_bilaplx_bilaplxp=<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[-141959.31]]], dtype=float32)>
analytic_bilaplx_bilaplxp=<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-141959.27]], dtype=float32)>
double_bilapl_rel_err=<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[3.3020035e-07]]], dtype=float32)>
